In [1]:
# Let's import required libs

import kagglehub
import pandas as pd
import numpy as np
import os
from pprint import pprint
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F


In [2]:
#path = kagglehub.dataset_download("ranadeep/credit-risk-dataset")

In [3]:
# We have to use PySpark since our file weight is > 400 mb, so Pandas is not useful
spark = SparkSession\
    .builder\
    .appName('Spark')\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/17 17:22:35 WARN Utils: Your hostname, mazer1x-v15xv17xrnx, resolves to a loopback address: 127.0.1.1; using 192.168.0.107 instead (on interface wlp0s20f3)
25/12/17 17:22:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/17 17:22:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('../data/loan/loan.csv',header=True) # our main dataset
columns_desc_pd = pd.read_excel('../data/LCDataDictionary.xlsx')
columns_describe = spark.createDataFrame(columns_desc_pd)

/home/mazer1x/miniconda3/envs/menv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## EDA

In [6]:
# let's check column types
columns = {}
for _,t in df.dtypes:
    if t not in columns: columns[t]=0
    columns[t]+=1
pprint(columns)

{'string': 74}


In [7]:
# % nans
nan_percentage = df.select([
    (F.round(
        F.sum(
            (F.col(c).isNull() | (F.col(c) == "NaN") | (F.col(c) == "")).cast("int")
        ) / df.count(),
        2
    )* 100).alias(c)
    for c in df.columns
])

In [8]:
rows_dict = (nan_percentage.collect()[0]).asDict()

25/12/17 17:22:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [9]:
# Columns with 10%+ nans//nulls
c = 0
columns_to_drop = []
for k,v in rows_dict.items(): 
    if v > 10: 
        c+=1
        columns_to_drop.append(k)
        print(k,v)

desc 52.0
mths_since_last_delinq 57.99999999999999
mths_since_last_record 89.0
next_pymnt_d 66.0
mths_since_last_major_derog 88.0
annual_inc_joint 100.0
dti_joint 100.0
verification_status_joint 100.0
tot_coll_amt 47.0
tot_cur_bal 47.0
open_acc_6m 100.0
open_il_6m 100.0
open_il_12m 100.0
open_il_24m 100.0
mths_since_rcnt_il 100.0
total_bal_il 100.0
il_util 100.0
open_rv_12m 100.0
open_rv_24m 100.0
max_bal_bc 100.0
all_util 100.0
total_rev_hi_lim 47.0
inq_fi 100.0
total_cu_tl 100.0
inq_last_12m 100.0


In [10]:
print(len(df.columns),c) # we can drop 25 cols. because of high nan %

74 25


In [11]:
df = df.drop(*columns_to_drop)

In [12]:
# Now lets remove unnecessary columns

In [13]:
for i in df.columns:
    print(i,df.select(i).distinct().count())
df.select("*").count()

id 91113
member_id 91113
loan_amnt 1095
funded_amnt 1155
funded_amnt_inv 9373
term 2
int_rate 448
installment 25056
grade 8
sub_grade 36
emp_title 53610
emp_length 13
home_ownership 6
annual_inc 9282
verification_status 4
issue_d 60
loan_status 10
pymnt_plan 3
url 91113
purpose 268
title 32200
zip_code 1076
addr_state 269
dti 3697
delinq_2yrs 228
earliest_cr_line 758
inq_last_6mths 219
open_acc 144
pub_rec 110
revol_bal 34069
revol_util 1257
total_acc 226
initial_list_status 157
out_prncp 25128
out_prncp_inv 25330
total_pymnt 86617
total_pymnt_inv 86075
total_rec_prncp 37763
total_rec_int 80117
total_rec_late_fee 2915
recoveries 6805
collection_recovery_fee 5129
last_pymnt_d 188
last_pymnt_amnt 60930
last_credit_pull_d 208
collections_12_mths_ex_med 149
policy_code 77
application_type 55
acc_now_delinq 41


91113

In [14]:
# Columns to drop, columns with (?) - extra check needed before drop.]
#id, member_id, emp_title, url, title(?), zip_code, delinq_2yrs(?), inq_last_6mths(?), open_acc(?), 
#initial_list_status(?), issue_d,pymnt_plan

# Post-issue
#out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee
#recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, collections_12_mths_ex_med,
#last_credit_pull_d

#first of all, lets drop post-issue columns.

In [15]:
post_issue_columns = ['out_prncp',
    'out_prncp_inv',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',
    'recoveries',
    'collection_recovery_fee',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'collections_12_mths_ex_med',
    'last_credit_pull_d'
] + [
    'id',
    'member_id',
    'emp_title',
    'url',
    'zip_code',
    'issue_d',
    'pymnt_plan'
]

In [16]:
df = df.drop(*post_issue_columns)

In [17]:
# Analysis of "problem" columns:

In [18]:
columns_to_cast = ['delinq_2yrs','inq_last_6mths','open_acc','initial_list_status']

In [19]:
for i in columns_to_cast:
    df1 = df.withColumn(i, F.col(i).try_cast('double'))
df1.select([
    F.count(
        F.when(
            F.col(c).isNull(), 1)
    ).alias(c)
    for c in columns_to_cast
]).show()

+-----------+--------------+--------+-------------------+
|delinq_2yrs|inq_last_6mths|open_acc|initial_list_status|
+-----------+--------------+--------+-------------------+
|         31|            31|     111|              90904|
+-----------+--------------+--------+-------------------+



In [20]:
# So, we can drop initial_list_status and remove columns with NULL-s: delinq_2yrs,inq_last_6mths,open_acc

In [21]:
df = df.drop('initial_list_status')

In [22]:
df = df.dropna() # data with NULL // NaN is less than 1% of the data
df.select([
    F.count(
        F.when(
            F.col(c).isNull(), 1)
    ).alias(c)
    for c in df.columns
]).show(vertical=True)

-RECORD 0------------------
 loan_amnt           | 0   
 funded_amnt         | 0   
 funded_amnt_inv     | 0   
 term                | 0   
 int_rate            | 0   
 installment         | 0   
 grade               | 0   
 sub_grade           | 0   
 emp_length          | 0   
 home_ownership      | 0   
 annual_inc          | 0   
 verification_status | 0   
 loan_status         | 0   
 purpose             | 0   
 title               | 0   
 addr_state          | 0   
 dti                 | 0   
 delinq_2yrs         | 0   
 earliest_cr_line    | 0   
 inq_last_6mths      | 0   
 open_acc            | 0   
 pub_rec             | 0   
 revol_bal           | 0   
 revol_util          | 0   
 total_acc           | 0   
 policy_code         | 0   
 application_type    | 0   
 acc_now_delinq      | 0   



In [23]:
# now we need to convert our data in needed types

In [24]:
df.show(1,vertical=True)

-RECORD 0--------------------------
 loan_amnt           | 5000.0      
 funded_amnt         | 5000.0      
 funded_amnt_inv     | 4975.0      
 term                |  36 months  
 int_rate            | 10.65       
 installment         | 162.87      
 grade               | B           
 sub_grade           | B2          
 emp_length          | 10+ years   
 home_ownership      | RENT        
 annual_inc          | 24000.0     
 verification_status | Verified    
 loan_status         | Fully Paid  
 purpose             | credit_card 
 title               | Computer    
 addr_state          | AZ          
 dti                 | 27.65       
 delinq_2yrs         | 0.0         
 earliest_cr_line    | Jan-1985    
 inq_last_6mths      | 1.0         
 open_acc            | 3.0         
 pub_rec             | 0.0         
 revol_bal           | 13648.0     
 revol_util          | 83.7        
 total_acc           | 9.0         
 policy_code         | 1.0         
 application_type    | INDIV

In [25]:
double_cols = ['loan_amnt','funded_amnt','funded_amnt_inv','int_rate','installment','annual_inc','dti','delinq_2yrs',
           'inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc','policy_code','acc_now_delinq']

In [26]:
for i in double_cols:
    df = df.withColumn(i, F.col(i).try_cast('double'))
df.select([
    F.count(
        F.when(
            F.col(c).isNull(), 1)
    ).alias(c)
    for c in double_cols
]).show(vertical=True)

-RECORD 0--------------
 loan_amnt       | 0   
 funded_amnt     | 0   
 funded_amnt_inv | 0   
 int_rate        | 0   
 installment     | 0   
 annual_inc      | 0   
 dti             | 54  
 delinq_2yrs     | 54  
 inq_last_6mths  | 54  
 open_acc        | 52  
 pub_rec         | 43  
 revol_bal       | 30  
 revol_util      | 41  
 total_acc       | 36  
 policy_code     | 7   
 acc_now_delinq  | 20  



In [27]:
df = df.dropna() # to drop nan-s

## Feature Engineering (FE) / String type

In [28]:
df.dtypes

[('loan_amnt', 'double'),
 ('funded_amnt', 'double'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'double'),
 ('verification_status', 'string'),
 ('loan_status', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('addr_state', 'string'),
 ('dti', 'double'),
 ('delinq_2yrs', 'double'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'double'),
 ('open_acc', 'double'),
 ('pub_rec', 'double'),
 ('revol_bal', 'double'),
 ('revol_util', 'double'),
 ('total_acc', 'double'),
 ('policy_code', 'double'),
 ('application_type', 'string'),
 ('acc_now_delinq', 'double')]

In [29]:
str_cols = []
for i in df.dtypes:
    if i[1] == 'string': str_cols+=[i[0]]

In [30]:
df.select(*str_cols).show(1,vertical=True)

-RECORD 0--------------------------
 term                |  36 months  
 grade               | B           
 sub_grade           | B2          
 emp_length          | 10+ years   
 home_ownership      | RENT        
 verification_status | Verified    
 loan_status         | Fully Paid  
 purpose             | credit_card 
 title               | Computer    
 addr_state          | AZ          
 earliest_cr_line    | Jan-1985    
 application_type    | INDIVIDUAL  
only showing top 1 row


In [31]:
# We should use GBT to solve our case, so we need to adapt our data.

In [33]:
# emp_length  - ordinal encoding
# term - ordinal encoding encoding
# grade + sub_grade - ordinal (check the cell below)
# home_ownership - OHE
# verification_status - ordinal encoding
# purpose - OHE
# title - Drop (31941 unique values)
# addr_state - Transform to (West/East/...) -> OHE
# earliest_cr_line - Int (From datetime to yeas in int format)
# application_type - INDIVIDUAL, only 1 distinct val, drop

In [34]:
#We can drop grade and use sub_grade instead. E5 = 1, E4 = 2, ..., A1 = 35

In [35]:
#df.select('loan_status').distinct().show()
#We can use Binary class. to exclude data leakage (instead of ternary сlassification)